In [1]:
import numpy as np

# Question 1 Vous devez tout d’abord écrire une structure de données d’arbre pondéré comme celui de la figure ci-dessus

In [2]:
class Graph:
    
    def __init__(self):
        self.poid = {} # Seule les arrêtes sont enregistrées
        self.value_functions = {} # Stockage des fonctions de valeur
    
    def weight(self,parent, etat_parent,child, etat_child, poid): # le couple (child,etat_child) reprente le noeud
        
        self.poid['(' + str(parent)+','+str(etat_parent)+')' +' <--> '+'(' +str(child)+','+str(etat_child)+')'] = poid
    def display(self):
        for k in self.poid.keys():
            print(k,':',self.poid[k])


In [3]:
g = Graph() # Initialisation du graphe
g.weight(0, 0, 1, 1, 1)
g.weight(0, 0, 1, 2, 5)
g.weight(0, 0, 1, 3,-3)
g.weight(1, 1, 2, 1,-2)
g.weight(1, 1, 2, 2, 5)
g.weight(1, 2, 2, 1, 2)
g.weight(1, 2, 2, 2, 7)
g.weight(1, 2, 2, 3, 4)
g.weight(1, 3, 2, 2, 7)
g.weight(1, 3, 2, 3, 3)
g.weight(2, 1, 3, 1, 3)
g.weight(2, 1, 3, 2, 5)
g.weight(2, 2, 3, 1,-2)
g.weight(2, 2, 3, 2, 1)
g.weight(2, 2, 3, 3, 2)
g.weight(2, 3, 3, 2, 0)
g.weight(2, 3, 3, 3, 4)
g.weight(3, 1, 4, 1, 0)
g.weight(3, 2, 4, 1, 0)
g.weight(3, 3, 4, 1, 0)

g.display()

(0,0) <--> (1,1) : 1
(0,0) <--> (1,2) : 5
(0,0) <--> (1,3) : -3
(1,1) <--> (2,1) : -2
(1,1) <--> (2,2) : 5
(1,2) <--> (2,1) : 2
(1,2) <--> (2,2) : 7
(1,2) <--> (2,3) : 4
(1,3) <--> (2,2) : 7
(1,3) <--> (2,3) : 3
(2,1) <--> (3,1) : 3
(2,1) <--> (3,2) : 5
(2,2) <--> (3,1) : -2
(2,2) <--> (3,2) : 1
(2,2) <--> (3,3) : 2
(2,3) <--> (3,2) : 0
(2,3) <--> (3,3) : 4
(3,1) <--> (4,1) : 0
(3,2) <--> (4,1) : 0
(3,3) <--> (4,1) : 0


# Question 2 Un premier programme déterminera par la méthode de récursivité inverse la fonction de valeur dans chaque état du graphe pour chaque niveau de l’arbre.

In [4]:
# Fonction permettant de recuperer les actions qui ont été prises pour atteindre un état spécifique à partir de l'état précédent
def recupe_action(child,etat_child):
    poid = list(g.poid.keys()) # Récupération de toutes les arrêtes 
    items = []
    for p in poid:
        if '-> (' + str(child)+','+str(etat_child)+')' in p :
            
        
            items.append(p)
    return items


In [5]:
recupe_action(1,1)

['(0,0) <--> (1,1)']

In [6]:
def V_etoile(child, etat_child):   # Cette fonction calcule la valeur de la fonction de valeur optimale V*(s) pour un état child et etat_child donnée
    if child == 0: 
        val = 0
    else:
        vals = []
        for p in recupe_action(child, etat_child):
            try:
                vals.append(g.poid[p] + V_etoile(int(p[1]), int(p[3]))) # Appel récursif de V_etoile
            except KeyError:    # Si une erreur KeyError est levée, cela signifie que l'action n'a pas été trouvée et on l'ignore 
                pass
        if vals:
            val = max(vals)
        else:
            val = float('-inf')
    g.value_functions['V*(k='+str(child)+',s='+str(etat_child)+')'] = val
   
    return val


In [7]:
V_etoile(4,1)
for k in g.value_functions.keys():
    print(k,'=',g.value_functions[k])


V*(k=0,s=0) = 0
V*(k=1,s=1) = 1
V*(k=1,s=2) = 5
V*(k=2,s=1) = 7
V*(k=1,s=3) = -3
V*(k=2,s=2) = 12
V*(k=3,s=1) = 10
V*(k=2,s=3) = 9
V*(k=3,s=2) = 13
V*(k=3,s=3) = 14
V*(k=4,s=1) = 14


# Question 3 Un second programme calculera la politique optimale et donc le chemin le plus long depuis la racine (k = 0) jusqu’aux feuilles (k = T).

In [8]:
def politique_optimale(k, s):   # cette fonction permet de determiner la politique optimale
    politique = [] # la politique optimale est stocké dans une liste
    while k != 0 or s != 0:
        recupe_act = recupe_action(k, s)  
        value_func = [g.value_functions['V*(k='+p[1]+',s='+p[3]+')'] for p in recupe_act] # recuperation de la fonction valeur
        
        value = value_func[0]
        noeud = recupe_act[0]
        for index, p in enumerate(value_func[1:], start = 1):
            if p > value:
                noeud = recupe_act[index]
                value = p
        k = int(noeud[1])
        s = int(noeud[3])
        politique.insert(0,'(k='+str(k)+',s='+str(s)+')')
    return politique


In [9]:
politique =politique_optimale(4,1)
print('Le chemin le plus long est : V*(k=4,s=1) =',g.value_functions['V*(k='+str(4)+',s='+str(1)+')'])
print('Voici le chemin : ')
for k in politique[:-1]:
    print(k)
    print('    | \n    v')
print(politique[len(politique)-1])


Le chemin le plus long est : V*(k=4,s=1) = 14
Voici le chemin : 
(k=0,s=0)
    | 
    v
(k=1,s=2)
    | 
    v
(k=2,s=2)
    | 
    v
(k=3,s=3)


# Question 4 Un premier programme correspond à l’algorithme de programmation dynamique par itération de la valeur.

In [10]:
import math

In [11]:
n = 3
m = 4
gamma = 0.9
start = np.array([2,0])  #point de depart de notre grille
directions = {'LEFT':np.array([0,-1]),'UP':np.array([-1,0]),'RIGHT':np.array([0,1]),'DOWN':np.array([1,0])} # variable contenant les vecteurs de deplacement pour chaque direction
actions = np.array(['LEFT','RIGHT','UP','DOWN'])  # nom des actions possibles

In [12]:
board = np.array([[ 0, 0,  0,  1],          
                    [ 0, math.nan,  0, -1],       #notre grille contient une valeur nan qui représente état inaccessible
                    [ 0,  0,  0,  0]])
board_prev = board.copy()
obstacles = [(1,1),(0,3),(1,3)]
print(board)

[[ 0.  0.  0.  1.]
 [ 0. nan  0. -1.]
 [ 0.  0.  0.  0.]]


In [13]:
def verifi_position(position):
    if position[0] < 0 or position[0] >= n or position[1] < 0 or position[1] >= m or math.isnan(board[position[0],position[1]]):
        return True
    else:
        return False

In [14]:
def probabilite_de_transition(position, action):  # on definit les probabilité de se deplacer de la position courante à la position suivante suivant une action donnée
    sum_pond = 0   # on initialise la somme pondéré des proba de transion
    next_position = position + directions[action]
     # on verifie si la position est au mure ou obstacle , la proba de transition est multiplié par 0.8
    if verifi_position(next_position):
        sum_pond = sum_pond + board_prev[position[0],position[1]]*0.8
    else:
        sum_pond = sum_pond + board_prev[next_position[0],next_position[1]]*0.8
    
    # On calcule les deux direction perpendiculaire à l'action selon c'est left right up or down    
    direction_perpendiculaire = []
    if action == 'LEFT' or action == 'RIGHT':
        direction_perpendiculaire.append('UP')
        direction_perpendiculaire.append('DOWN')
    else:
        direction_perpendiculaire.append('LEFT')
        direction_perpendiculaire.append('RIGHT')
            
    
    for a in direction_perpendiculaire:
        next_position = position + directions[a]
        if verifi_position(next_position):
            sum_pond = sum_pond + board_prev[position[0],position[1]]*0.1
        else:
            sum_pond = sum_pond + board_prev[next_position[0],next_position[1]]*0.1
    return sum_pond

In [15]:
def value_iteration(position):
    return  gamma*max([probabilite_de_transition(position,a) for a in actions])

In [16]:
for i in range(5):   # on verifie les valeurs de la probabilité de transion pour 5 itération
    for k in range(n):
        for l in range(m):
            if (k,l) not in obstacles:
                board[k,l] = round(value_iteration([k,l]),3)
    board_prev = board.copy()
print(board)

[[ 0.585  0.734  0.845  1.   ]
 [ 0.413    nan  0.565 -1.   ]
 [ 0.214  0.306  0.43   0.188]]


# Question 5 Un second programme correspond à la détermination de la politique optimale par itération de la politique.

In [17]:
def policy_iteration(position):
    probabilite_de_transition_list = [probabilite_de_transition(position,a) for a in actions]
    probabilite_de_transition_max = max(probabilite_de_transition_list)
    policy = actions[np.argmax(probabilite_de_transition_list)]
    return  gamma*probabilite_de_transition_max, policy

def init_policy_board():
    policy_board = []
    for k in range(n):
        policy_board.append([])
        for l in range(m): 
            policy_board[k].append("     ")
    policy_board = np.array(policy_board)      
    policy_board[0,3] = 'UP'
    policy_board[1,3] = 'LEFT'
    policy_board_prev = policy_board.copy()
    
    return policy_board, policy_board_prev

In [18]:
policy_board, policy_board_prev = init_policy_board()
it = 1
while True:
    for k in range(n):
        for l in range(m):
            if (k,l) not in obstacles:
                result = policy_iteration([k,l])
                board[k,l] = round(result[0],3)
                policy_board[k,l]= result[1]
    board_prev = board.copy()
    it += 1
    if np.array_equal(policy_board,policy_board_prev):
        break
    policy_board_prev = policy_board.copy()
print(policy_board,'\n')
print('Nombre d iteration est : ',it)

[['RIGHT' 'RIGHT' 'RIGHT' 'UP']
 ['UP' '     ' 'UP' 'LEFT']
 ['UP' 'RIGHT' 'UP' 'LEFT']] 

Nombre d iteration est :  3
